In [15]:
from zlib import crc32

import numpy as np
import matplotlib.pylab as plt
from matplotlib import gridspec
import pandas as pd
import scipy.stats as st
import seaborn as sns
import itertools
from itertools import combinations
from  statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [3]:
#Генерируем выборку
import re
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    size = rs.randint(low=5, high=10)
    data = rs.randn(30, size)
    for i in range(size):
        data[:, i] += rs.uniform()*2-1.0
    return data

In [4]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

Enter your mail: salnikov.an@phystech.edu


In [12]:
data = task['data']
#Для удобной работы со столбцами
ndata = data.T

In [20]:
#t-test о сравнении средних всех пар из столбцов выборки
def test(ndata):
    rejections = 0
    p_values = []
    for x, y in combinations(ndata, 2):
        p_value = st.ttest_ind(x, y)[1]
        p_values.append(p_value)
        rejections += p_value <= 0.05
    return rejections, p_values
print('Rejections = {}'.format(test(ndata)[0]))

Rejections = 17


In [28]:
#Уровень значимости при использовании поправки Бонферрони
print('P_value = {}'.format(multipletests(test(ndata)[1], method='bonferroni')[-1])) 

P_value = 0.0017857142857142859


In [29]:
#Кол-во пар из столбцов, для которых отвергается гипотеза о совпадении среднего при использовании поправки Бонферрони
print('Rejections = {}'.format(multipletests(test(ndata)[1], method='bonferroni')[0].sum()))

Rejections = 14


In [30]:
#Кол-во пар из столбцов, для которых отвергается гипотеза о совпадении среднего при использовании метода Холма
print('Rejections = {}'.format(multipletests(test(ndata)[1], method='holm')[0].sum()))

Rejections = 14


In [32]:
#Оценка m0 по методу Шведера-Спьётволла (lambda = 0.5)
l = 0.5
m0 = (sum([1 for x in test(ndata)[1] if x > l]) + 1) / (1 - l)
print('m0 = {}'.format(m0))

m0 = 6.0
